In [1]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle as pkl
pkl_file = "/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/Main-Implementation/model_save/trend_pred"

In [2]:
df = pd.read_csv('/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/trend_data/trend_emotion_mapping.csv')

In [3]:
dataset = df.iloc[:, 2:]

In [4]:
from tqdm import tqdm

trends = ["approval","toxic","obscene","threat","hate", "offensive", "neutral"]
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in tqdm(range(df.shape[0])):
        temp = [0]*7
        labels = df.iloc[i]["trend_type"][1:-2].split(', ')
        # print(labels)
        for label in labels:
            # print(label.replace('\'', ''))
            idx = trends.index(label.replace('\'', ''))
            temp[idx] = 1
        one_hot_encoding.append(temp)
    return pd.DataFrame(one_hot_encoding)

In [5]:
ohe_labels = one_hot_encoder(dataset)

100%|██████████| 110209/110209 [00:06<00:00, 16753.29it/s]


In [6]:
featurs = dataset.iloc[:, 1:-7]
target = dataset.iloc[:, -7:]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(featurs, target, test_size=0.2, random_state=42)

## Random Forest

In [8]:
from sklearn.model_selection import cross_val_score

randomForest = RandomForestClassifier(criterion='entropy', max_depth=None, max_features=28, max_leaf_nodes=19, min_samples_leaf=28)
randomForest.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous-multioutput'

In [ ]:
y_pred = randomForest.predict(X_test)
y_pred

In [ ]:
import os
pickle = pkl.dump(randomForest, open(os.path.join(pkl_file, 'random_forest.pickle'), 'wb'))

In [ ]:
# rf_pkl = pkl.load(open(os.path.join(pkl_file, 'random_forest.pickle'), 'rb'))
# y_pred = rf_pkl.predict(X_test)
# y_pred

In [ ]:
from sklearn.metrics import classification_report
cr_y = classification_report(y_test, y_pred)
print(cr_y)

## Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(f"Predicted response:\n{y_pred[0]}")

In [ ]:
import os
pickle = pkl.dump(lr, open(os.path.join(pkl_file, 'linear_reg.pickle'), 'wb'))

In [ ]:
lg_pkl = pkl.load(open(os.path.join(pkl_file, 'linear_reg.pickle'), 'rb'))
y_pred = lg_pkl.predict(X_test)
print(f"Predicted response:\n{y_pred[0]}")

In [ ]:
r_squared = lr.score(X_train, y_train) # Getting model score for test dataset
print(f"Model Score:\n{r_squared}") # Showing the model score

## Logistic Regression

In [ ]:
# using classifier chains
from sklearn.metrics import accuracy_score
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

# initialize classifier chains multi-label classifier
log_reg = ClassifierChain(LogisticRegression(C= 0.001, penalty= 'l2'))
# print(log_reg.get_params().keys())
# Training logistic regression model on train data
log_reg.fit(X_train, y_train)
# predict
y_pred = log_reg.predict(X_test)
print(y_pred[0])
# accuracy
print("Accuracy = ", accuracy_score(y_test, y_pred))
print("\n")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import numpy as np
# define model
grid={"classifier__C": np.logspace(-3,3,7), "classifier__penalty": ["l1","l2"]} # l1 lasso l2 ridge
logreg = ClassifierChain(LogisticRegression())
logreg_cv = GridSearchCV(logreg, grid, cv=5)
# define search
result = logreg_cv.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
import os
pickle = pkl.dump(log_reg, open(os.path.join(pkl_file, 'log_reg.pickle'), 'wb'))

In [ ]:
import numpy as np
lg_pkl = pkl.load(open(os.path.join(pkl_file, 'log_reg.pickle'), 'rb'))
y_pred = lg_pkl.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
cr_y = classification_report(y_test, y_pred)
print(cr_y)

In [ ]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
import warnings
warnings.filterwarnings('ignore')
# initialize label powerset multi-label classifier
# classifier = LabelPowerset(LogisticRegression())
grid={"classifier__C": np.logspace(-3,3,7), "classifier__penalty": ["l1","l2"]} # l1 lasso l2 ridge
logreg1 = LabelPowerset(LogisticRegression())
print(logreg1.get_params().keys())
logreg_cv = GridSearchCV(logreg, grid, cv=5)
# train
logreg_cv.fit(X_train, y_train)
# predict
predictions = logreg_cv.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

## ML-KNN

In [ ]:
# from skmultilearn.adapt import MLkNN
# from scipy.sparse import csr_matrix, lil_matrix
# classifier_new = MLkNN(k=10)
# # Note that this classifier can throw up errors when handling sparse matrices.
# x_train = lil_matrix(X_train).toarray()
# y_train = lil_matrix(y_train).toarray()
# x_test = lil_matrix(X_test).toarray()
# # train
# classifier_new.fit(X_train, y_train)
# # predict
# predictions_new = classifier_new.predict(x_test)
# # accuracy
# print("Accuracy = ",accuracy_score(y_test,predictions_new))
# print("\n")

## Support Vector Machine

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import warnings
warnings.filterwarnings('ignore')

# # defining parameter range 
param_grid={"estimator__C": np.logspace(-3,3,7), "estimator__penalty": ["l1","l2"]} # l1 lasso l2 ridge

ln_svc = LinearSVC(random_state=42)
multilabel_classifier = MultiOutputClassifier(estimator=ln_svc, n_jobs=-1)
print(multilabel_classifier.get_params().keys())

   
grid = GridSearchCV(multilabel_classifier, param_grid, verbose = 2, cv=5) 
   
# fitting the model for grid search 
grid.fit(X_train, y_train)
 
# print best parameter after tuning
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
param_grid={"estimator__C": np.logspace(-3,3,7), "estimator__penalty": ["l1","l2"]} # l1 lasso l2 ridge
ln_svc = LinearSVC(random_state=42)
svmClassifier = OneVsRestClassifier(ln_svc, n_jobs=-1)
print(svmClassifier.get_params().keys())

grid = GridSearchCV(svmClassifier, param_grid, verbose = 2, cv=5) 
# fitting the model for grid search 
grid.fit(X_train, y_train)
 
# print best parameter after tuning
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 

In [ ]:
from skmultilearn.problem_transform import LabelPowerset

powerSetSVC = LabelPowerset(LinearSVC())
print(svmClassifier.get_params().keys())
powerSetSVC.fit(X_train, y_train)

## Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

boostClassifier = OneVsRestClassifier(GradientBoostingClassifier())
boostClassifier.fit(X_train, y_train)
boostPreds = boostClassifier.predict(X_test)

In [ ]:
print(classification_report(y_test, boostPreds)) 

## Naive Bayes

In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
params = {'classifier__alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
          'classifier__fit_prior': [True, False],
          'classifier__class_prior': [None, [0.1,]*(7) ]
         }
nb_classifier = BinaryRelevance(MultinomialNB())
print(nb_classifier.get_params().keys())

multinomial_nb_grid = GridSearchCV(nb_classifier, param_grid=params, n_jobs=-1, cv=10, verbose=1)
# train
multinomial_nb_grid.fit(X_train, y_train)
# predict
y_pred = multinomial_nb_grid.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test, y_pred))

In [ ]:
import os
pickle = pkl.dump(multinomial_nb_grid, open(os.path.join(pkl_file, 'naive_bayes.pickle'), 'wb'))

In [ ]:
from sklearn.metrics import classification_report
cr_y = classification_report(y_test, y_pred)
print(cr_y)

In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
nb_classifier = ClassifierChain(MultinomialNB())
print(nb_classifier.get_params().keys())

multinomial_nb_grid = GridSearchCV(nb_classifier, param_grid=params, n_jobs=-1, cv=10, verbose=1)

# train
multinomial_nb_grid.fit(X_train, y_train)

# predict
predictions = multinomial_nb_grid.predict(X_test)

accuracy_score(y_test, predictions)

In [ ]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
nb_classifier = LabelPowerset(MultinomialNB())

print(nb_classifier.get_params().keys())

multinomial_nb_grid = GridSearchCV(nb_classifier, param_grid=params, n_jobs=-1, cv=10, verbose=1)

# train
multinomial_nb_grid.fit(X_train, y_train)

# predict
predictions = multinomial_nb_grid.predict(X_test)

accuracy_score(y_test,predictions)

## RandomForest with GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
rf = RandomForestClassifier()
grid_search  = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
import os
pickle = pkl.dump(grid_search, open(os.path.join(pkl_file, 'random_forest_grid.pickle'), 'wb'))

In [ ]:
import numpy as np

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: ', np.mean(errors))
    print('Accuracy = ', accuracy)
    
    return accuracy

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
yhat = best_grid.predict(X_test)
acc = accuracy_score(y_test, yhat)
print("Accuracy: ", acc)
cr_y = classification_report(y_test, yhat)
print(cr_y)

## XGB Classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier())

clf = Pipeline([('classify', classifier)]) 
clf.fit(X_train, y_train)
print("Training Acc: ",clf.score(X_train, y_train))

In [ ]:
import os
pickle = pkl.dump(clf, open(os.path.join(pkl_file, 'xgb_classifier.pickle'), 'wb'))

In [ ]:
yhat = clf.predict(X_test)
acc = accuracy_score(y_test, yhat)
print("Accuracy: ", acc)

In [ ]:
cr_y1 = classification_report(y_test, yhat)
print(cr_y1)

In [ ]:
# # write code to calculate the R squared error
# from sklearn.metrics import r2_score
# r_squared_score = r2_score(y_test, y_pred) # Calculating model R^2 error

In [ ]:
#view R-squared value
# print(f"R squared error:\n{r_squared_score}") # Showing the R squared error

## Multi-Label-Classification using MLP

In [ ]:

# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
 

In [ ]:
# get the model
def get_model(n_inputs, n_outputs):
 model = Sequential()
 model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(n_outputs, activation='sigmoid'))
 model.compile(loss='binary_crossentropy', optimizer='sgd')
 return model

In [ ]:
import os
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)
    # enumerate folds
    # for train_ix, test_ix in cv.split(X):
        # prepare data
    # X_train, X_test = X[train_ix], X[test_ix]
    # y_train, y_test = y[train_ix], y[test_ix]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, verbose=1, epochs=10)

    pickle = pkl.dump(model, open(os.path.join(pkl_file, 'mlp_classifier.pickle'), 'wb'))
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    # store result
    print('>%.3f' % acc)
    # results.append(acc)
    return acc, yhat

In [ ]:
# load dataset
# X, y = get_dataset()
# evaluate model
acc, yhat = evaluate_model(X_train, y_train)
# summarize performance
# print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

In [ ]:
cr_y = classification_report(y_test, yhat)
print(cr_y)